In [1]:
# source: https://ogb.stanford.edu/docs/nodeprop/#ogbn-arxiv

from ogb.nodeproppred import NodePropPredDataset

dataset = NodePropPredDataset(name = "ogbn-arxiv")

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph, label = dataset[0]

Using backend: pytorch


Downloaded 0.08 GB: 100%|██████████| 81/81 [00:14<00:00,  5.44it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 4236.67it/s]

Saving...


In [2]:
import json
import numpy as np
import torch
import scipy.sparse as sparse


In [4]:
node_feats = graph['node_feat'] # (169343, 128)
node_class = label.reshape(-1,)  # (169343,)
node_year = graph['node_year']
edge = graph['edge_index'].T # (1166243, 2)
node_list = np.ones((1, graph['num_nodes']))
edge_list = np.ones((1, edge.shape[0]))

data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "node_year":node_year,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

In [5]:
np.savez_compressed("ogbn-arxiv.npz", **data)


In [6]:
cora = np.load("ogbn-arxiv.npz", allow_pickle=True)
cora.files

['node_feats', 'node_class', 'node_year', 'edge', 'node_list', 'edge_list']

In [9]:
task_data = {
    "train": train_idx,
    "val": valid_idx,
    "test": test_idx
}
np.savez_compressed("ogbn-arxiv_task.npz", **task_data)